In [52]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:////workspaces/mlops-zoomcamp/02-experiment-tracking/mlflow.db"

client = MlflowClient(tracking_uri= MLFLOW_TRACKING_URI)

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
s =client.get_experiment_by_name("nyc-taxi-experiment1")
print(s.artifact_location)
print(s.experiment_id)

/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2
2


In [3]:
client.create_experiment(name = "My_cool_experiment",
                         artifact_location= "/workspaces/mlops-zoomcamp/02-experiment-tracking/mlflow_artifacts" )

'3'

In [43]:
from mlflow.entities import ViewType
runs = client.search_runs(
experiment_ids='2',filter_string='metrics.rmse<5.2', run_view_type = ViewType.ACTIVE_ONLY, max_results=5, order_by=['metrics.rmse ASC']
)

In [44]:
for run in runs:
    print(f"Run Name: {runs[0].data.tags['mlflow.runName']} \nRun ID: {run.info.run_id} \nRMSE: {run.data.metrics['rmse']}")
    print("-----------------------------------")

Run Name: gentle-panda-933 
Run ID: 8d70334c2fb548eaaade87579707b133 
RMSE: 5.142496528432483
-----------------------------------
Run Name: gentle-panda-933 
Run ID: 0338e611743c4663b78fece083312bab 
RMSE: 5.144916069308788
-----------------------------------


In [57]:
model_name = "nyc_taxi_regressor"
versions = client.get_latest_versions(name = model_name)
for sv in versions:
    print(f"Current Stage: {sv.current_stage} \nVersion: {sv.version} \nModel_Algo:{sv.tags['Model']} \nRun ID:{sv.run_id}")
    print("----------------------------------------")

Current Stage: Production 
Version: 1 
Model_Algo:Regressor GB 
Run ID:0338e611743c4663b78fece083312bab
----------------------------------------
Current Stage: Staging 
Version: 2 
Model_Algo:XGB 
Run ID:47f6b87d87db416592332e2664952a59
----------------------------------------


In [78]:
sv

<ModelVersion: aliases=[], creation_timestamp=1716390300945, current_stage='Staging', description='', last_updated_timestamp=1716390512640, name='nyc_taxi_regressor', run_id='47f6b87d87db416592332e2664952a59', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/47f6b87d87db416592332e2664952a59/artifacts/models_mlflow', status='READY', status_message=None, tags={'Model': 'XGB'}, user_id=None, version=2>

In [72]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import mlflow


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [74]:
df = read_dataframe("/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-03.parquet")

In [67]:
client.download_artifacts( run_id = "47f6b87d87db416592332e2664952a59",
#path = "/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/47f6b87d87db416592332e2664952a59/artifacts/preprocessor",
path = 'preprocessor',
dst_path=".")

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [68]:
import pickle

with open("/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [75]:
%time
X_test = preprocess(df, dv)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [76]:
target = 'duration'
y_test = df[target].values

In [79]:
%time test_model(name = "nyc_taxi_regressor", stage = "Staging", X_test = X_test, y_test = y_test )

MlflowException: Registered Model with name=nyc_taxi_regressor not found

In [81]:
mlflow.pyfunc.load_model(model_uri = "/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/0338e611743c4663b78fece083312bab/artifacts/model")

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 0338e611743c4663b78fece083312bab

In [ ]:
mlflow.pyfunc.load_model()

In [ ]:
model_uri: The location, in URI format, of the MLflow model. For example:

        - ``/Users/me/path/to/local/model``
        - ``relative/path/to/local/model``
        - ``s3://my_bucket/path/to/model``
        - ``runs:/<mlflow_run_id>/run-relative/path/to/model``
        - ``models:/<model_name>/<model_version>``
        - ``models:/<model_name>/<stage>``
        - ``mlflow-artifacts:/path/to/model``

        For more information about supported URI schemes, see
        `Referencing Artifacts <https://www.mlflow.org/docs/latest/concepts.html#
        artifact-locations>`_.
    suppress_warnings: If ``True``, non-fatal warning messages associated with the model
        loading process will be suppressed. If ``False``, these warning messages will be
        emitted.
    dst_path: The local filesystem path to which to download the model artifact.
        This directory must already exist. If unspecified, a local output
        path will be created.
    model_config: The model configuration to apply to the model. This configuration
        is available during model loading.